In [10]:
from extract import extract
from extract_all import extract_all
import PyPDF2 as p
import config as conf
import os

In [2]:
df = extract('../test_data/pdf/01-IGACOS2010_Audit_Report.pdf')

56-75
../test_data/pdf/01-IGACOS2010_Audit_Report.pdf
../test_data/pdf/01-IGACOS2010_Audit_Report.pdf


https://unix.stackexchange.com/questions/533005/moving-a-file-based-on-specific-filename-to-specific-folder-based-on-filename

In [3]:
df

,audit observation,recommendations,ref,management action,status of implementation,reasons for partial/non-implementation,city,year
0,Non-settingupofRPT/SET Receivables1.RealProper...,DirecttheCityTreasurer to submitaCertifiedLi...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
1,UnrecordedInterest Income2. Failure to recor...,DirecttheCityAccountant to takeupthenecessar...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
2,NaN,statements.,NaN,NaN,NaN,NaN,IGACOS,2010
3,AuditObservations (CY 2009),Recommendations,Ref,ManagementAction,Status ofImplementation,Reason forPartial/Non-Implementation,IGACOS,2010
4,Understated InsuranceExpense. The expired p...,DirecttheCityAccountanttoreview allprepaymenta...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
5,Uncancelled StaleChecks4.Stalechecksamounting ...,DirecttheCityAccountanttoregularly review th...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
6,AuditObservations (CY 2009),Recommendations,Ref,ManagementAction,Status ofImplementation,Reason forPartial/Non-Implementation,IGACOS,2010
7,Unadjusted InventoryAccount With No StockBalan...,DirecttheCityAccountant to takeupthefollowin...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
8,Unreconciled Differencein Inventory Reports6.T...,DirecttheCityAccountant and theCityTreasurer...,NaN,Implemented,100%implemented.,NaN,IGACOS,2010
9,AuditObservations (CY 2009),Recommendations,Ref,ManagementAction,Status ofImplementation,Reason forPartial/Non-Implementation,IGACOS,2010


In [4]:
df = extract_all('../test_data/pdf')

56-75
../test_data/pdf/01-IGACOS2010_Audit_Report.pdf
../test_data/pdf/01-IGACOS2010_Audit_Report.pdf
1-3
../test_data/pdf/02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.pdf
../test_data/pdf/02-LamitanCity2010_Status_of_Prior_Years_Audit_Recommendations.pdf
1-34
../test_data/pdf/10-ManilaCity2021_Part3-Status_of_PYs_Recomm.pdf
../test_data/pdf/10-ManilaCity2021_Part3-Status_of_PYs_Recomm.pdf


In [5]:
df

,audit observation,recommendations,references,status of implementation,reasons for partial/non-implementation,management action,observations and recommendations,ref,city,year
0,Non-settingupofRPT/SET Receivables1.RealProper...,DirecttheCityTreasurer to submitaCertifiedLi...,NaN,100%implemented.,NaN,Implemented,NaN,NaN,IGACOS,2010
1,UnrecordedInterest Income2. Failure to recor...,DirecttheCityAccountant to takeupthenecessar...,NaN,100%implemented.,NaN,Implemented,NaN,NaN,IGACOS,2010
2,NaN,statements.,NaN,NaN,NaN,NaN,NaN,NaN,IGACOS,2010
3,AuditObservations (CY 2009),Recommendations,NaN,Status ofImplementation,Reason forPartial/Non-Implementation,ManagementAction,NaN,Ref,IGACOS,2010
4,Understated InsuranceExpense. The expired p...,DirecttheCityAccountanttoreview allprepaymenta...,NaN,100%implemented.,NaN,Implemented,NaN,NaN,IGACOS,2010
...,...,...,...,...,...,...,...,...,...,...
200,NaN,NaN,NaN,NaN,NaN,NaN,We recommended thatManagement:,NaN,ManilaCity,2021
201,NaN,NaN,Reference,Status ofImplementation,Reason forPartial / Non-Implementation,ManagementAction,Observations andRecommendations,NaN,ManilaCity,2021
202,NaN,NaN,NaN,Not Implemented,Modification andrepairoftheCTISMhasnotbeencoo...,Lack of effectiveactionbytheconcernedofficials.,a.RequiretheconcernedCityofficialstocoordinate...,NaN,ManilaCity,2021
203,NaN,NaN,NaN,Not Implemented,Extensive trainingforthehospitalpersonnel has...,Lack of effectiveactionbytheconcernedofficials.,b.Makeproperrepresentationwiththe contractor f...,NaN,ManilaCity,2021


In [11]:
def test_scrapability(): # IGACOS causes problem in test of convert.sh
        """ We need all location-year combinations represented in output (convert.sh) """
        """ Issues of lack of output are caused by unconventional formatting """
        # all location-year combos present in /unzipped/
            # if not, then problem in FILENAME_TARGET
        cities_years = {
        'zipd':[filename.replace('-',' ').replace('Annual Audit Report','').replace('.zip','').split('  ') for filename in os.listdir(self.zipd)],
        'pdf':[(filename[2:].split('_'))[0].replace('-','') for filename in os.listdir(self.pdf) if filename.split('.')[1] == 'pdf']
        }
        for city_year in cities_years['zipd']:
            self.assertIn((city_year[0]+city_year[1]).replace(' ',''), cities_years['pdf'])
        # all location-year combos present in combined dataframe
            # if not, then problem in TARGET_SENTENCE
        for city_year in cities_years['zipd']:
            self.assertIn(city_year[0].join(city_year[1]).replace(' ',''), extract_all(self.pdf)['city']+extract_all(self.pdf)['year'])

In [12]:
test_scrapability()

NameError: name 'self' is not defined